In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import torch

import tsvar

import experiments_utils

# Set numpy print format
np.set_printoptions(precision=2, floatmode='fixed', sign=' ')

# Set experiments verbosity
#experiments_utils.PRINT_EVERY = 1
#experiments_utils.CALLBACK_END = ''
PRINT_EVERY = 1
CALLBACK_END = ''
VI_N_ITER = 1000

In [16]:
dim = 2
max_jumps = int(5e3 * dim)
gen_seed = 1234
sim_seed = 789

print('Generate parameters:')
print('--------------------')
coeffs_true_dict = experiments_utils.generate_parameters(dim=dim, seed=gen_seed,
                                                         base_range=[0.1, 0.1], adj_range=[0.2, 0.3])
coeffs_true_dict = {k: np.array(v) for k, v in coeffs_true_dict.items()}
for k,v in coeffs_true_dict.items():
    print(f"{k}:")
    print(np.array(v))
print()

print('Simulate data:')
print('--------------')
events, end_time, sim_seed = experiments_utils.generate_data(**coeffs_true_dict, max_jumps=max_jumps)
print(f'Simulated {sum(map(len, events)):d} events')

Generate parameters:
--------------------
baseline:
[ 0.10  0.10]
beta:
[[ 0.44  0.79]
 [ 0.78  0.27]]
adjacency:
[[ 0.24  0.00]
 [ 0.00  0.00]]

Simulate data:
--------------
Simulated 10000 events


In [38]:
# Extract ground truth
coeffs_true = np.hstack((coeffs_true_dict['baseline'],
                         coeffs_true_dict['adjacency'].flatten()))

# Set model
model = tsvar.models.WoldModelVariationalFixedBeta(verbose=True)
model.observe(events, beta=coeffs_true_dict['beta'])

# Set priors
as_pr = 1.0 * np.ones((dim + 1, dim))
ar_pr = 1.0 * np.ones((dim + 1, dim))
zc_pr = [1.0 * np.ones((len(events[i]), dim+1)) for i in range(dim)]

# Set callback (parameters of callback are just the posterior mean of alpha)
callback = tsvar.utils.callbacks.LearnerCallbackMLE(
    x0=(as_pr / ar_pr).flatten(), print_every=10,
    coeffs_true=coeffs_true_dict['adjacency'].flatten(),
    acc_thresh=0.05, dim=dim, default_end='')

# Fit model
conv = model.fit(as_pr=as_pr, ar_pr=ar_pr, zc_pr=zc_pr, max_iter=1000,
                 tol=1e-5, callback=callback)

iter:   220 | dx: +1.0134e-05 | acc: 1.00 | f1-score: 1.00 | relerr: 0.05 | time/it: 7.79e-03     

In [39]:
model._as_po / model._ar_po

array([[ 0.09,  0.10],
       [ 0.26,  0.01],
       [ 0.00,  0.01]])

In [40]:
model._as_po

array([[ 3910.33,  4043.37],
       [ 1918.11,    51.92],
       [   24.56,    57.71]])

In [41]:
model._ar_po

array([[ 41250.99,  41268.28],
       [  7363.37,   7709.55],
       [  6615.71,   7518.51]])

In [42]:
model.D_ikj[0]

array([[  2.09,   0.00,   0.00],
       [  1.98,   0.00,   0.00],
       [  2.63,   0.77,   1.12],
       ...,
       [  2.04,   0.20,   0.20],
       [ 32.55,   9.35,   2.63],
       [  1.11,   0.03,   0.56]])

In [43]:
model._zp_po[0]

array([[ 1.00,  0.00,  0.00],
       [ 1.00,  0.00,  0.00],
       [ 0.55,  0.44,  0.01],
       ...,
       [ 0.79,  0.21,  0.00],
       [ 0.56,  0.44,  0.00],
       [ 0.91,  0.07,  0.02]])

In [44]:
np.hstack((model.events[0][0], np.diff(model.events[0])))

array([  2.09,   1.98,   2.63, ...,   2.04,  32.55,   1.11], dtype=float32)